In [23]:
from mhadatareader import MhaDataReader
from classes import ParticipantsData, Scan, ProficiencyLabel, FoldSplit
import utils as ut
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import itertools as it
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [24]:
DIR_NAME = './data_bckp'
MODEL_NAME = 'best_model.tf'
LOG_DIR = './logs'

In [25]:
def load_data():
    parser = MhaDataReader()
    intermediates = parser.read_data(f'{DIR_NAME}/Intermediates/')
    sanity_check(intermediates)

    ut.add_path_len(intermediates)
    ut.add_linear_speed(intermediates)
    ut.add_angular_speed(intermediates)

    x_intermed = ut.prepare_data_all_reg(intermediates)

    experts = parser.read_data(f'{DIR_NAME}/Experts/')
    sanity_check(experts)

    ut.add_path_len(experts)
    ut.add_linear_speed(experts)
    ut.add_angular_speed(experts)
    x_expert = ut.prepare_data_all_reg(experts)

    novices = parser.read_data(f'{DIR_NAME}/Novices/')
    sanity_check(novices)

    ut.add_path_len(novices)
    ut.add_linear_speed(novices)
    ut.add_angular_speed(novices)
    x_novice = ut.prepare_data_all_reg(novices)

    return x_novice, x_intermed, x_expert


def sanity_check(data: ParticipantsData):
    for part in data:
        assert len(part.get_transforms(Scan.ALL)) == \
               len(part.get_transforms(Scan.LUQ)) + len(part.get_transforms(Scan.RUQ)) + \
               len(part.get_transforms(Scan.PERICARD)) + len(part.get_transforms(Scan.PELVIC))

        assert part.get_time() == part.get_reg_time(Scan.RUQ) + part.get_reg_time(Scan.LUQ) + \
               part.get_reg_time(Scan.PERICARD) + part.get_reg_time(Scan.PELVIC)

In [26]:
novices, intermed, experts = load_data()
all = novices + intermed + experts
slice_window = 500
print(slice_window)

folds = ut.form_folds(novices, intermed, experts)
folds_stats = []
models_train_hist = dict()

500


In [30]:
iterations = list(it.permutations([i for i in range(len(folds))]))
print(folds[:1])

[([array([[ 3.69170364e-02,  3.26238328e-02,  3.56920519e-02, ...,
        -3.48646203e-01, -4.26243377e-01, -4.92525511e-01],
       [-9.98749252e-01, -9.98775738e-01, -9.98567392e-01, ...,
        -8.89944721e-01, -8.48595364e-01, -8.10121805e-01],
       [ 3.37203692e-02,  3.71848208e-02,  3.98652867e-02, ...,
        -2.94013978e-01, -3.13372769e-01, -3.17995726e-01],
       ...,
       [ 0.00000000e+00,  6.62285002e-02,  2.17320685e-01, ...,
         1.21020909e+00,  1.29257506e+00,  1.27165132e+00],
       [ 0.00000000e+00,  4.16830510e+01,  1.02217901e+01, ...,
         6.19703411e-01,  7.24707900e-01,  7.28064358e-01],
       [ 0.00000000e+00,  9.95571429e-02,  1.98871429e-01, ...,
         3.37909286e+01,  3.38908714e+01,  3.39906571e+01]]), array([[ 1.06939219e-01,  1.07094800e-01,  9.55614981e-02, ...,
        -3.62482924e-01, -4.79038032e-01, -5.71619104e-01],
       [-9.94082610e-01, -9.94117238e-01, -9.95328812e-01, ...,
        -8.94989039e-01, -8.48771552e-01, -8.036216

In [47]:
train, valid, test = iterations[0]
train, valid, test = folds[train], folds[valid], folds[test]
train, valid, test = ut.prepare_folds(train, valid, test, slice_window)

x_train, y_train = train
x_val, y_val = valid
x_test, y_test = test

In [48]:
print(x_train.shape[1:])

(16, 450)


In [94]:
def build_model(dropout_rate=0.4, filters=8):
    input_shape=(16, 450)
    num_classes=3
    #filters=8
    kernel_size=5
    #dropout_rate=0.7
    learning_rate=0.0001
    regularizer = keras.regularizers.l1_l2(0.2)
    
    input_layer = keras.layers.Input(shape=input_shape)

    conv1 = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding="same", kernel_regularizer=regularizer)(input_layer)
    conv1 = keras.layers.Dropout(dropout_rate)(conv1)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding="same", kernel_regularizer=regularizer)(conv1)
    conv2 = keras.layers.Dropout(dropout_rate)(conv2)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding="same", kernel_regularizer=regularizer)(conv2)
    conv3 = keras.layers.Dropout(dropout_rate)(conv3)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)
    output_layer = keras.layers.Dense(num_classes, activation='softmax')(gap)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    
    optimizer = keras.optimizers.Adam(
        learning_rate=learning_rate,
    )
    
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['val_loss']
    )
    return model

In [95]:
model = KerasClassifier(build_fn=build_model, epochs=80, batch_size=32, verbose=1)
batch_size = [16, 32, 64, 128]
epochs = [100, 130, 150, 200, 250, 300]
learning_rate = [0.0001, 0.001, 0.02, 0.05]
kernel_size = [3, 5, 7, 9]
filters = [16, 32, 64]
dropout_rate = [0.3, 0.5, 0.6]
param_grid = dict(
    #batch_size=batch_size, 
    #epochs=epochs,
    #learning_rate=learning_rate,
    #kernel_size=kernel_size,
    filters=filters,
    dropout_rate=dropout_rate
)
grid = GridSearchCV(
    estimator=model, 
    param_grid=param_grid, n_jobs=3, cv=3, verbose=1)

In [91]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
grid_result = grid.fit(x_train, y_train, verbose=1)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  27 out of  27 | elapsed: 54.2min finished


Epoch 1/80
535/535 [==============================] - 6s 10ms/step - loss: 707.1181 - accuracy: 0.4772
Epoch 2/80
535/535 [==============================] - 5s 9ms/step - loss: 45.3144 - accuracy: 0.8561
Epoch 3/80
535/535 [==============================] - 5s 10ms/step - loss: 1.3485 - accuracy: 0.9137
Epoch 4/80
535/535 [==============================] - 5s 10ms/step - loss: 1.3545 - accuracy: 0.9138
Epoch 5/80
535/535 [==============================] - 5s 10ms/step - loss: 1.3174 - accuracy: 0.9202
Epoch 6/80
535/535 [==============================] - 5s 10ms/step - loss: 1.3343 - accuracy: 0.9192
Epoch 7/80
535/535 [==============================] - 5s 10ms/step - loss: 1.3526 - accuracy: 0.9172
Epoch 8/80
535/535 [==============================] - 6s 10ms/step - loss: 1.3434 - accuracy: 0.9252
Epoch 9/80
535/535 [==============================] - 5s 10ms/step - loss: 1.3735 - accuracy: 0.9242
Epoch 10/80
535/535 [==============================] - 5s 10ms/step - loss: 1.3854 - accu

In [92]:

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.963116 using {'dropout_rate': 0.5, 'filters': 64}
0.804409 (0.131963) with: {'dropout_rate': 0.3, 'filters': 16}
0.955110 (0.021325) with: {'dropout_rate': 0.3, 'filters': 32}
0.782028 (0.089138) with: {'dropout_rate': 0.3, 'filters': 64}
0.937341 (0.053216) with: {'dropout_rate': 0.5, 'filters': 16}
0.945642 (0.047659) with: {'dropout_rate': 0.5, 'filters': 32}
0.963116 (0.012819) with: {'dropout_rate': 0.5, 'filters': 64}
0.918751 (0.033148) with: {'dropout_rate': 0.6, 'filters': 16}
0.929742 (0.026286) with: {'dropout_rate': 0.6, 'filters': 32}
0.919513 (0.024428) with: {'dropout_rate': 0.6, 'filters': 64}


In [93]:
f = open("grid_filter_dropout.txt", "a")
f.write(str(grid_result.cv_results_))
f.close()